In [ ]:
import pandas as pd

import emission.core.get_database as edb
import emission.core.wrapper.entry as ecwe
import emission.storage.decorations.analysis_timeseries_queries as esda
import emission.storage.decorations.trip_queries as esdt
import emission.storage.decorations.timeline as esdl
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.timeseries.timequery as estt
import scaffolding
from uuid import UUID

%matplotlib inline

In [ ]:
year = None
month = None
program = "default"
study_type = "program"
mode_of_interest = "e-bike"
include_test_users = True

## Load the programs

In [ ]:
# Split UUIDs by program
program_uuid_map = {}
for ue in edb.get_uuid_db().find():
    program = ue['user_email'].split("_")[0]
    if program in program_uuid_map.keys():
        program_uuid_map[program].append(str(ue['uuid']))
    else:
        print(f"Found new program {program}, creating new list")
        program_uuid_map[program] = []
        program_uuid_map[program].append(str(ue['uuid']))

In [ ]:
uuid_program_list = []
for ue in edb.get_uuid_db().find():
    program = ue['user_email'].split("_")[0]
    uuid_program_list.append({"program": program, "opcode": ue["user_email"], "user_id_str": str(ue['uuid'])})

In [ ]:
uuid_program_df = pd.DataFrame.from_dict(uuid_program_list)
uuid_program_df.head()

In [ ]:
from collections import defaultdict

In [ ]:
# Loading mapping dictionaries from mapping_dictionaries notebook
%store -r dic_re
%store -r dic_pur

# convert a dictionary to a defaultdict
dic_re = defaultdict(lambda: 'Other',dic_re)
dic_pur = defaultdict(lambda: 'Other',dic_pur)

## Load the trip table

In [ ]:
expanded_ct, file_suffix, quality_text, debug_df = scaffolding.load_viz_notebook_data(year,
                                                                            month,
                                                                            program,
                                                                            study_type,
                                                                            dic_re,
                                                                            dic_pur=dic_pur,
                                                                            include_test_users=include_test_users)

In [ ]:
expanded_ct["user_id_str"] = expanded_ct.user_id.apply(lambda u: str(u))
len(expanded_ct.user_id_str.unique())

## Combine program and trip tables

In [ ]:
trip_program_df = expanded_ct.merge(uuid_program_df, on="user_id_str")

In [ ]:
trip_program_df["distance_km"] = trip_program_df.distance / 1000

In [ ]:
trip_program_df.sample(n=50, random_state=123)[["program", "user_id_str", "opcode", "_id", "start_fmt_time", "end_fmt_time", "distance_km", "mode_confirm"]]

In [ ]:
%conda install geopandas

## Load shape files

In [ ]:
import geopandas as gpd

In [ ]:
denver_boundary = gpd.read_file("geom/denver_uza.shp")

In [ ]:
# denver_boundary.to_json()

In [ ]:
denver_boundary.plot()

In [ ]:
denver_pixels = gpd.read_file("geom/denver_uza_grid.shp")

In [ ]:
denver_pixels

In [ ]:
denver_pixels.plot()

## Prepping dataframes for plotting

In [ ]:
all_start_end_points = trip_program_df.start_loc.append(trip_program_df.end_loc)

In [ ]:
import shapely as shp

In [ ]:
# I wanted to use shapely's from_geojson but it doesn't seem to be supported in this version
all_geo_start_end_points = gpd.GeoSeries(all_start_end_points.apply(lambda p: shp.geometry.Point(p['coordinates'])), crs="EPSG:4269")

In [ ]:
denver_boundary.geometry.iloc[0]

In [ ]:
all_start_or_end_within = all_geo_start_end_points[all_geo_start_end_points.within(denver_boundary.geometry.iloc[0])]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax_arr = plt.subplots(nrows=1, ncols=2)
denver_boundary.plot(color="#E66D0A", ax=ax_arr[0])
denver_boundary.plot(color="#E66D0A", ax=ax_arr[1])
all_start_or_end_within.plot(ax=ax_arr[1])

In [ ]:
e_bike_trips = trip_program_df[trip_program_df.mode_confirm == 'pilot_ebike']; len(e_bike_trips)

In [ ]:
car_like_trips = trip_program_df.query('mode_confirm == "drove_alone" | mode_confirm == "shared_ride" | mode_confirm == "taxi"'); len(car_like_trips)

In [ ]:
e_bike_start_end_points = e_bike_trips.start_loc.append(e_bike_trips.end_loc); len(e_bike_start_end_points)

In [ ]:
e_bike_geo_start_end_points = gpd.GeoSeries(e_bike_start_end_points.apply(lambda p: shp.geometry.Point(p['coordinates'])), crs="EPSG:4269")

In [ ]:
e_bike_start_or_end_within = e_bike_geo_start_end_points[e_bike_geo_start_end_points.within(denver_boundary.geometry.iloc[0])]

In [ ]:
car_like_start_end_points = car_like_trips.start_loc.append(car_like_trips.end_loc); len(car_like_start_end_points)

In [ ]:
car_like_geo_start_end_points = gpd.GeoSeries(car_like_start_end_points.apply(lambda p: shp.geometry.Point(p['coordinates'])), crs="EPSG:4269")

In [ ]:
car_like_start_or_end_within = car_like_geo_start_end_points[car_like_geo_start_end_points.within(denver_boundary.geometry.iloc[0])]

### General plotting of start/end points

In [ ]:
fig, ax_arr = plt.subplots(nrows=1, ncols=3, figsize=(12,6), sharex=True, sharey=True)
denver_boundary.boundary.plot(ax=ax_arr[0])
all_start_or_end_within.plot(ax=ax_arr[0])
denver_boundary.boundary.plot(ax=ax_arr[1])
e_bike_start_or_end_within.plot(color="#28a745", ax=ax_arr[1])
denver_boundary.boundary.plot(ax=ax_arr[2])
car_like_start_or_end_within.plot(color="#dc3545", ax=ax_arr[2])

### Heatmap using folium

Example from: https://stackoverflow.com/a/65756840/4040267

In [ ]:
import folium
import folium.plugins as fpl

In [ ]:
denver_boundary.representative_point()

In [ ]:
import numpy as np

In [ ]:
all_start_or_end_within.apply(lambda p: [p.y, p.x])

In [ ]:
np.array([all_start_or_end_within.x, all_start_or_end_within.y])

In [ ]:
map = folium.Map(
    location=(39.67707, -104.96168),
    zoom_start=11
)
fpl.HeatMap(all_start_or_end_within.apply(lambda p: [p.y, p.x])).add_to(map)
map

In [ ]:
import branca.element as bre

In [ ]:
all_map = folium.Map(
    location=(39.67707, -104.96168),
    zoom_start=11
)
fpl.HeatMap(all_start_or_end_within.apply(lambda p: [p.y, p.x])).add_to(all_map)

e_bike_map = folium.Map(
    location=(39.67707, -104.96168),
    zoom_start=11
)
fpl.HeatMap(e_bike_start_or_end_within.apply(lambda p: [p.y, p.x])).add_to(e_bike_map)

car_like_map = folium.Map(
    location=(39.67707, -104.96168),
    zoom_start=11
)
fpl.HeatMap(car_like_start_or_end_within.apply(lambda p: [p.y, p.x])).add_to(car_like_map)


fig = bre.Figure()
fig.add_subplot(1,3,1).add_child(all_map)
fig.add_subplot(1,3,2).add_child(e_bike_map)
fig.add_subplot(1,3,3).add_child(car_like_map)

### Heatmap using geopandas

Example from: https://gist.github.com/perrygeo/c426355e40037c452434

In [ ]:
from scipy import ndimage

In [ ]:
denver_bounds_extent = (denver_boundary.bounds)[["minx", "maxx", "maxy", "miny"]].values.tolist()[0]

In [ ]:
def heatmap(d, bins=(100,100), smoothing=1.3, cmap='viridis', ax=None, bounds=None):
    heatmap, xedges, yedges = np.histogram2d(d.y, d.x, bins=bins)
    if bounds is None:
        extent = [yedges[0], yedges[-1], xedges[-1], xedges[0]]
        print(extent)
    else:
        extent = bounds
        print(extent)

    logheatmap = np.log(heatmap)
    logheatmap[np.isneginf(logheatmap)] = 0
    logheatmap = ndimage.filters.gaussian_filter(logheatmap, smoothing, mode='nearest')
    # heatmap = ndimage.filters.gaussian_filter(heatmap, smoothing, mode='nearest')
    
    if ax is None:
        plt.imshow(logheatmap, cmap=cmap, extent=extent)
        plt.colorbar()
        plt.gca().invert_yaxis()
        plt.show()
    else:
        ax.imshow(logheatmap, cmap=cmap, extent=extent)
        ax.invert_yaxis()

In [ ]:
heatmap(all_start_or_end_within)

In [ ]:
fig, ax_arr = plt.subplots(nrows=1, ncols=3, sharex=True, sharey=True, figsize=(12,6))
denver_boundary.boundary.plot(ax=ax_arr[0])
heatmap(all_start_or_end_within, ax=ax_arr[0], bounds=denver_bounds_extent)
denver_boundary.boundary.plot(ax=ax_arr[1])
heatmap(e_bike_start_or_end_within, ax=ax_arr[1], bounds=denver_bounds_extent)
denver_boundary.boundary.plot(ax=ax_arr[2])
heatmap(car_like_start_or_end_within, ax=ax_arr[2], bounds=denver_bounds_extent)

### Using seaborn

https://www.geeksforgeeks.org/kde-plot-visualization-with-pandas-and-seaborn/

In [ ]:
import seaborn as sns

In [ ]:
ax = sns.kdeplot(x=all_start_or_end_within.x, y=all_start_or_end_within.y, color='g', shade=True)
denver_boundary.boundary.plot(ax=ax)

In [ ]:
fig, ax_arr = plt.subplots(nrows=1, ncols=3, sharex=True, sharey=True, figsize=(12,6))
denver_boundary.boundary.plot(ax=ax_arr[0])
sns.kdeplot(x=all_start_or_end_within.x, y=all_start_or_end_within.y, ax=ax_arr[0], color='g', shade=True)
denver_boundary.boundary.plot(ax=ax_arr[1])
sns.kdeplot(x=e_bike_start_or_end_within.x, y=e_bike_start_or_end_within.y, ax=ax_arr[1], color='g', shade=True)
denver_boundary.boundary.plot(ax=ax_arr[2])
sns.kdeplot(x=car_like_start_or_end_within.x, y=car_like_start_or_end_within.y, ax=ax_arr[2], color='g', shade=True)

In [ ]:
all_geo_start_end_points.head()

### Plot all e-bike trajectories (note that this will take several hours)

In [ ]:
def get_points_for_trip(user_id, trip_id):
    ts = esta.TimeSeries.get_time_series(user_id)
    return ts.get_data_df("analysis/recreated_location",
                         time_query=esda.get_time_query_for_trip_like("analysis/confirmed_trip", trip_id))

In [ ]:
# e_bike_all_trip_points = e_bike_trips.iloc[0:5].apply(lambda t: get_points_for_trip(t.user_id, t._id), axis=1)
e_bike_all_trip_points = e_bike_trips.apply(lambda t: get_points_for_trip(t.user_id, t._id), axis=1)

In [ ]:
import functools

In [ ]:
e_bike_all_points = functools.reduce(lambda x, y: x.append(y), e_bike_all_trip_points)

In [ ]:
e_bike_all_points["loc"]

In [ ]:
e_bike_all_points_gs = gpd.GeoSeries(e_bike_all_points["loc"].apply(lambda p: shp.geometry.Point(p['coordinates'])), crs="EPSG:4269")

In [ ]:
e_bike_all_points_within = e_bike_all_points_gs[e_bike_all_points_gs.within(denver_boundary.geometry.iloc[0])]

In [ ]:
fig, ax_arr = plt.subplots(nrows=1, ncols=1, figsize=(24,12))
# durham_boundary.plot(color="#E66D0A", ax=ax_arr[0])
denver_boundary.boundary.plot(ax=ax_arr)
e_bike_all_points_within.plot(ax=ax_arr, markersize=1)

### Final, most complicated version using pixels, and computing the ratio of e-bike to car_like trips

In [ ]:
# First, let's just make a dataframe with the three different counts: total, e-bike, car-like, for each polygon
def get_counts(pixel_polygon):
    all_trip_count = np.count_nonzero(all_geo_start_end_points.within(pixel_polygon))
    e_bike_trip_count = np.count_nonzero(e_bike_geo_start_end_points.within(pixel_polygon))
    car_like_trip_count = np.count_nonzero(car_like_geo_start_end_points.within(pixel_polygon))
    return pd.Series([all_trip_count, e_bike_trip_count, car_like_trip_count])

#### Test with a small dataset

In [ ]:
test_pixels = denver_pixels.head(n=50).copy()

In [ ]:
test_pixels

In [ ]:
test_pixels[["all_trip_count", "e_bike_trip_count", "car_like_trip_count"]] = test_pixels.geometry.apply(lambda pp: get_counts(pp))

In [ ]:
test_pixels.head()

In [ ]:
fig, ax_arr = plt.subplots(nrows=3, ncols=1, figsize=(12,6), sharex=True, sharey=True)
test_pixels.plot(column="all_trip_count", legend=True, ax=ax_arr[0])
test_pixels.plot(column="e_bike_trip_count", legend=True, ax=ax_arr[1])
test_pixels.plot(column="car_like_trip_count", legend=True, ax=ax_arr[2])

In [ ]:
test_pixels["e_bike_2_car_like"] = test_pixels.e_bike_trip_count / test_pixels.car_like_trip_count

In [ ]:
test_pixels.head()

In [ ]:
test_pixels.e_bike_2_car_like.dropna()

In [ ]:
ax = denver_boundary.boundary.plot()
test_pixels.plot(column="e_bike_2_car_like", ax=ax, legend=True)

### And now for the final version

In [ ]:
denver_pixels_all = denver_pixels.copy()
denver_pixels_all[["all_trip_count", "e_bike_trip_count", "car_like_trip_count"]] = denver_pixels_all.geometry.apply(lambda pp: get_counts(pp))

In [ ]:
fig, ax_arr = plt.subplots(nrows=1, ncols=3, figsize=(12,6), sharex=True, sharey=True)
denver_pixels_all.plot(column="all_trip_count", ax=ax_arr[0], cmap="YlOrRd")
denver_pixels_all.plot(column="e_bike_trip_count", ax=ax_arr[1], cmap="YlOrRd")
denver_pixels_all.plot(column="car_like_trip_count", ax=ax_arr[2], cmap="YlOrRd")

In [ ]:
denver_pixels_all["e_bike_2_car_like"] = denver_pixels_all.e_bike_trip_count / denver_pixels_all.car_like_trip_count

In [ ]:
denver_pixels_all.replace(np.inf, denver_pixels_all.replace(np.inf, 0).e_bike_2_car_like.max(), inplace=True)

In [ ]:
denver_pixels_all[denver_pixels_all.e_bike_2_car_like > 5]

In [ ]:
ax = denver_boundary.boundary.plot()
denver_pixels_all.plot(column="e_bike_2_car_like", ax=ax, cmap="YlGn", legend=True)

In [ ]:
denver_pixels_all.e_bike_2_car_like.dropna().plot(kind="hist", bins=35)

In [ ]:
np.count_nonzero(denver_pixels_all.e_bike_2_car_like < 0.5), np.count_nonzero(denver_pixels_all.e_bike_2_car_like >= 0.5)

In [ ]:
108 * 100 / 658

In [ ]:
denver_pixels_all["e_bike_better"] = pd.Categorical(denver_pixels_all.e_bike_2_car_like >= 1)

In [ ]:
ax = denver_boundary.boundary.plot()
denver_pixels_all.dropna(axis='rows', how='any').plot(column = "e_bike_better", ax = ax, legend=True, legend_kwds={"loc": "lower left"}, categorical=True, cmap="Paired", figsize=(12,6))

In [ ]:
denver_pixels_all.dropna().groupby("e_bike_better").count().id.plot(kind="pie", autopct='%.1f', title="Pixels where e-bikes are used more often than driving")

In [ ]:
denver_pixels_all["e_bike_gt_50_pct"] = pd.Categorical(denver_pixels_all.e_bike_2_car_like >= 0.5)

In [ ]:
denver_pixels_all.dropna().groupby("e_bike_gt_50_pct").count().id.plot(kind="pie", autopct='%.1f', title="Pixels where e-bike trips are least 50% of driving trips")

## Another complex check - does this differ across programs

In [ ]:
def get_endpoints_within(trip_df):
    start_end_points = trip_df.start_loc.append(trip_df.end_loc)
    geo_start_end_points = gpd.GeoSeries(start_end_points.apply(lambda p: shp.geometry.Point(p['coordinates'])), crs="EPSG:4269")
    start_or_end_within = geo_start_end_points[geo_start_end_points.within(denver_boundary.geometry.iloc[0])]
    return start_or_end_within

In [ ]:
def get_all_e_bike_car_like(trip_df):
    return {"all": get_endpoints_within(trip_df),
            "e_bike": get_endpoints_within(trip_df.query('mode_confirm == "pilot_ebike"')),
            "car_like": get_endpoints_within(trip_df.query('mode_confirm == "drove_alone" | mode_confirm == "shared_ride" | mode_confirm == "taxi"'))
           }

In [ ]:
sc_trips_split = get_all_e_bike_car_like(trip_program_df.query("program == 'sc'"))

In [ ]:
prepilot_trips_split = get_all_e_bike_car_like(trip_program_df.query("program == 'prepilot'"))

In [ ]:
def get_kde_plots(trips_split):
    fig, ax_arr = plt.subplots(nrows=1, ncols=3, sharex=True, sharey=True, figsize=(12,6))
    denver_boundary.boundary.plot(ax=ax_arr[0])
    sns.kdeplot(x=trips_split["all"].x, y=trips_split["all"].y, ax=ax_arr[0], color='g', shade=True)
    denver_boundary.boundary.plot(ax=ax_arr[1])
    sns.kdeplot(x=trips_split["e_bike"].x, y=trips_split["e_bike"].y, ax=ax_arr[1], color='g', shade=True)
    denver_boundary.boundary.plot(ax=ax_arr[2])
    sns.kdeplot(x=trips_split["car_like"].x, y=trips_split["car_like"].y, ax=ax_arr[2], color='g', shade=True)
    # return fig

In [ ]:
get_kde_plots(sc_trips_split)

In [ ]:
get_kde_plots(prepilot_trips_split)

In [ ]:
def get_scatter_plots(trips_split):
    fig, ax_arr = plt.subplots(nrows=1, ncols=3, sharex=True, sharey=True, figsize=(12,6))
    denver_boundary.boundary.plot(ax=ax_arr[0])
    trips_split["all"].plot(ax=ax_arr[0])
    ax_arr[0].set_title("all trips")
    denver_boundary.boundary.plot(ax=ax_arr[1])
    trips_split["e_bike"].plot(color="#28a745", ax=ax_arr[1])
    ax_arr[1].set_title("e-bike trips")
    denver_boundary.boundary.plot(ax=ax_arr[2])
    trips_split["car_like"].plot(color="#dc3545", ax=ax_arr[2])
    ax_arr[2].set_title("car trips")
    # return fig

In [ ]:
get_scatter_plots(sc_trips_split)

In [ ]:
get_scatter_plots(prepilot_trips_split)

In [ ]:
# First, let's just make a dataframe with the three different counts: total, e-bike, car-like, for each polygon
def get_counts_trips_split(trips_split, pixel_polygon):
    all_trip_count = np.count_nonzero(trips_split["all"].within(pixel_polygon))
    e_bike_trip_count = np.count_nonzero(trips_split["e_bike"].within(pixel_polygon))
    car_like_trip_count = np.count_nonzero(trips_split["car_like"].within(pixel_polygon))
    return pd.Series([all_trip_count, e_bike_trip_count, car_like_trip_count])

In [ ]:
def get_pixel_stats(trips_split):
    curr_pixel_stats = denver_pixels.copy()
    curr_pixel_stats[["all_trip_count", "e_bike_trip_count", "car_like_trip_count"]] = curr_pixel_stats.geometry.apply(lambda pp: get_counts_trips_split(trips_split, pp))
    curr_pixel_stats["e_bike_2_car_like"] = curr_pixel_stats.e_bike_trip_count / curr_pixel_stats.car_like_trip_count
    curr_pixel_stats["e_bike_better"] = pd.Categorical(curr_pixel_stats.e_bike_2_car_like >= 1)
    curr_pixel_stats["e_bike_gt_50_pct"] = pd.Categorical(curr_pixel_stats.e_bike_2_car_like >= 0.5)
    return curr_pixel_stats

In [ ]:
sc_pixel_stats = get_pixel_stats(sc_trips_split)

In [ ]:
def show_pixel_stats(pixel_stats):
    fig, ax_arr = plt.subplots(nrows=1, ncols=2)
    denver_boundary.boundary.plot(ax=ax_arr[0])
    pixel_stats.dropna(axis='rows', how='any').plot(column = "e_bike_better", ax = ax_arr[0], legend=True, legend_kwds={"loc": "lower left"}, categorical=True, cmap="Paired", figsize=(12,6))
    denver_boundary.boundary.plot(ax=ax_arr[1])
    pixel_stats.dropna(axis='rows', how='any').plot(column = "e_bike_gt_50_pct", ax = ax_arr[1], legend=True, legend_kwds={"loc": "lower left"}, categorical=True, cmap="Paired", figsize=(12,6))    

In [ ]:
show_pixel_stats(sc_pixel_stats)

In [ ]:
def show_pixel_percents(pixel_stats):
    fig, ax_arr = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True, figsize=(10,5))
    pixel_stats.dropna().groupby("e_bike_better").count().id.plot(kind="pie", ax=ax_arr[0], autopct='%.1f', title="e-bikes are used more often than driving")
    pixel_stats.dropna().groupby("e_bike_gt_50_pct").count().id.plot(kind="pie", ax=ax_arr[1], autopct='%.1f', title="e-bike trips are least 50% of driving trips")
    ax_arr[0].set_ylabel("% of pixels")

In [ ]:
show_pixel_percents(sc_pixel_stats)

In [ ]:
prepilot_pixel_stats = get_pixel_stats(prepilot_trips_split)

In [ ]:
show_pixel_stats(prepilot_pixel_stats)

In [ ]:
show_pixel_percents(prepilot_pixel_stats)

In [ ]:
other_programs_split = get_all_e_bike_car_like(trip_program_df.query('program != "sc" & program != "prepilot" & program != "stage"'))

In [ ]:
get_scatter_plots(other_programs_split)

In [ ]:
other_programs_pixel_stats = get_pixel_stats(other_programs_split)

In [ ]:
show_pixel_stats(other_programs_pixel_stats)

In [ ]:
show_pixel_percents(other_programs_pixel_stats)

## Experimenting with whether we can associate other fields with the start_end_loc points

In [ ]:
start_loc_df = trip_program_df[["program", "mode_confirm", "start_loc"]].copy()
start_loc_df["type"] = ["start"] * len(start_loc_df)
start_loc_df.rename(columns = {"start_loc": "location"}, inplace=True)
start_loc_df.head()

In [ ]:
end_loc_df = trip_program_df[["program", "mode_confirm", "end_loc"]].copy()
end_loc_df["type"] = ["end"] * len(end_loc_df)
end_loc_df.rename(columns = {"end_loc": "location"}, inplace=True)
end_loc_df.head()

In [ ]:
loc_program_df = gpd.GeoDataFrame(start_loc_df.append(end_loc_df))
loc_program_geodf = loc_program_df.set_geometry(loc_program_df['location'].apply(lambda p: shp.geometry.Point(p['coordinates'])), crs="EPSG:4269")
loc_program_geodf.head()

In [ ]:
program_endpoints_within = loc_program_geodf[loc_program_geodf.within(denver_boundary.geometry.iloc[0])]

In [ ]:
program_endpoints_within.head()

In [ ]:
fig, ax_arr = plt.subplots(nrows=1, ncols=3, sharex=True, sharey=True, figsize=(12,6))
denver_boundary.boundary.plot(ax=ax_arr[0])
program_endpoints_within.plot(column="program", cmap="tab10", ax=ax_arr[0])
denver_boundary.boundary.plot(ax=ax_arr[1])
program_endpoints_within.query('mode_confirm == "pilot_ebike"').plot(column="program", cmap="tab10", ax=ax_arr[1])
denver_boundary.boundary.plot(ax=ax_arr[2])
program_endpoints_within.query('mode_confirm == "drove_alone" | mode_confirm == "shared_ride" | mode_confirm == "taxi"').plot(column="program", cmap="tab10", ax=ax_arr[2])

In [ ]:
other_programs_endpoints_within_query = program_endpoints_within.query('program != "sc" & program != "prepilot" & program != "stage"')

In [ ]:
fig, ax_arr = plt.subplots(nrows=1, ncols=5, sharex=True, sharey=True, figsize=(20,10))
denver_boundary.boundary.plot(ax=ax_arr[0])
other_programs_endpoints_within_query.plot(column="program", cmap="tab10", categories=other_programs_endpoints_within_query.program.unique(), ax=ax_arr[0], legend=True, legend_kwds={"loc": "lower left"})
ax_arr[0].set_title("all")
denver_boundary.boundary.plot(ax=ax_arr[1])
other_programs_endpoints_within_query.query('mode_confirm == "pilot_ebike"').plot(column="program", cmap="tab10", categories=other_programs_endpoints_within_query.program.unique(), ax=ax_arr[1], legend=True, legend_kwds={"loc": "lower left"})
ax_arr[1].set_title("e-bike")
denver_boundary.boundary.plot(ax=ax_arr[2])
other_programs_endpoints_within_query.query('mode_confirm == "drove_alone"').plot(column="program", cmap="tab10", categories=other_programs_endpoints_within_query.program.unique(), ax=ax_arr[2], legend=True, legend_kwds={"loc": "lower left"})
ax_arr[2].set_title("drove_alone")
denver_boundary.boundary.plot(ax=ax_arr[3])
other_programs_endpoints_within_query.query('mode_confirm == "shared_ride"').plot(column="program", cmap="tab10", categories=other_programs_endpoints_within_query.program.unique(), ax=ax_arr[3], legend=True, legend_kwds={"loc": "lower left"})
ax_arr[3].set_title("shared_ride")
denver_boundary.boundary.plot(ax=ax_arr[4])
other_programs_endpoints_within_query.query('mode_confirm == "taxi"').plot(column="program", cmap="tab10", categories=other_programs_endpoints_within_query.program.unique(), ax=ax_arr[4], legend=True, legend_kwds={"loc": "lower left"})
ax_arr[4].set_title("taxi")
# ax_arr[0].legend(["denver"] + other_programs_endpoints_within_query.program.unique().tolist(), loc="lower left", bbox_to_anchor=(0.5, 0.5))

In [ ]:
program_endpoints_within.to_file("ceo_results/program_endpoints_within_denver")

In [ ]:
program_endpoints_within.to_csv("ceo_results/program_endpoints_within_denver.csv")

In [ ]:
sc_pixel_stats_to_export = sc_pixel_stats.copy()

In [ ]:
sc_pixel_stats_to_export = sc_pixel_stats_to_export.astype({"e_bike_better": bool, "e_bike_gt_50_pct": bool})

In [ ]:
sc_pixel_stats_to_export.to_file("ceo_results/sc_pixel_stats")

In [ ]:
prepilot_pixel_stats_to_export = prepilot_pixel_stats.copy()

In [ ]:
prepilot_pixel_stats_to_export = prepilot_pixel_stats_to_export.astype({"e_bike_better": bool, "e_bike_gt_50_pct": bool})

In [ ]:
prepilot_pixel_stats_to_export.to_file("ceo_results/prepilot_pixel_stats")

In [ ]:
sc_pixel_stats.to_csv("ceo_results/sc_pixel_stats.csv")

In [ ]:
prepilot_pixel_stats.to_csv("ceo_results/prepilot_pixel_stats.csv")

In [ ]:
other_programs_pixel_stats_to_export = other_programs_pixel_stats.copy()

In [ ]:
other_programs_pixel_stats_to_export = other_programs_pixel_stats_to_export.astype({"e_bike_better": bool, "e_bike_gt_50_pct": bool})

In [ ]:
other_programs_pixel_stats_to_export.to_file("ceo_results/other_programs_pixel_stats")

In [ ]:
other_programs_pixel_stats.to_csv("ceo_results/other_programs_pixel_stats.csv")